In [1]:
import os
import pathlib

from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import load_prompt
from langchain_community.llms.ollama import Ollama
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

# Get directory for python docs
__file__ = "/Users/prithvirajchaudhuri/Desktop/Other/Projects/jaac/notebook-examples/agent.ipynb"
directory = pathlib.Path(os.path.dirname(__file__)).parent
python_directory = directory / "docs"

# Create LLM instance
llm = Ollama(model="llama2", temperature=0.5, verbose=True)

# Specify toolkits
list_directory_tool, read_file_tool = FileManagementToolkit(
    root_dir=str(python_directory),
    selected_tools=["list_directory", "read_file"]
).get_tools()
internet_search_tool = DuckDuckGoSearchRun()
files = list_directory_tool.run(tool_input=".")
tools = [list_directory_tool, read_file_tool]

# Pull the prompt template
prompt = load_prompt(directory / "conf" / "custom_react_prompt.yaml")

# Create ReAct agent
react_agent = create_react_agent(
    llm = llm,
    tools = tools,
    prompt = prompt,
)
react_agent_exec = AgentExecutor(agent=react_agent, tools=tools, verbose=True, max_iterations=5, handle_parsing_errors=True, return_intermediate_steps=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/taagarwa/Projects/jaac-test/conf/custom_react_prompt.yaml'

In [11]:
question = "What does the new_function function in sample.py do?"
query = {"input": question, 'files': files}
react_agent_exec.invoke(query)



> Entering new AgentExecutor chain...
Great! Let's get started.

Question: What does the `new_function` function in `sample.py` do?
Thought: I'm not sure, let me look at the code.
Action: read_file
Action Input: sample.pyimport os

def addNumbers(a, b):
    sum = a + b
    return sum

def printEnv():
    print(os.getenv("TEST"))


def new_fuction():
    printEnv()
    result = addNumbers(10, 12)
    return result
Question: What does the `new_function` function in `sample.py` do?
Thought: Let me see...
Action: read_file
Action Input: sample.pyimport os

def addNumbers(a, b):
    sum = a + b
    return sum

def printEnv():
    print(os.getenv("TEST"))


def new_fuction():
    printEnv()
    result = addNumbers(10, 12)
    return result
Great! Let's answer the question together using the files in the directory.

Question: What does the `new_function` function in `sample.py` do?

Thought: I'm not sure, let me look at the code.
Action: read_file
Action Input: sample.pyimport os

def addNu

{'input': 'What does the new_function function in sample.py do?',
 'files': 'sample.py\ntest.py',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='read_file', tool_input='sample.py', log="Great! Let's get started.\n\nQuestion: What does the `new_function` function in `sample.py` do?\nThought: I'm not sure, let me look at the code.\nAction: read_file\nAction Input: sample.py"),
   'import os\n\ndef addNumbers(a, b):\n    sum = a + b\n    return sum\n\ndef printEnv():\n    print(os.getenv("TEST"))\n\n\ndef new_fuction():\n    printEnv()\n    result = addNumbers(10, 12)\n    return result\n'),
  (AgentAction(tool='read_file', tool_input='sample.py', log='Question: What does the `new_function` function in `sample.py` do?\nThought: Let me see...\nAction: read_file\nAction Input: sample.py'),
   'import os\n\ndef addNumbers(a, b):\n    sum = a + b\n    return sum\n\ndef printEnv():\n    print(os.getenv("TEST"))\n\n\ndef new_fuction(